In [1]:
import pandas as pd 

## Exhaustive

In [2]:
model_names = ['gemma-3-1b-pt', 'gpt2']

In [3]:
for model in model_names:

    filename_fn = lambda x: f'{x}-sample-stats.csv'
    df = pd.read_csv(f'../data/exhaustive/{filename_fn(model)}')

    print(df.head())

   sample_idx        count          mean          std        min           max
0           0  34359869440   9491.273973  2122.655164  28.347353  46543.382812
1           1  34359869440   9338.912643  1939.911406  27.463047  38455.750000
2           2  34359869440  10061.169184  1784.916624  26.331551  48011.203125
3           3  34359869440  10436.978361  2685.334255  43.648937  52808.609375
4           4  34359869440   9646.630449  2012.337411  29.853027  38111.019531
   sample_idx       count        mean        std       min          max
0           0  1262908153  181.936430  41.492789  3.429164  1129.703613
1           1  1262908153  168.808854  38.830127  4.335365  1112.325928
2           2  1262908153  166.570910  41.829423  3.037109   997.642029
3           3  1262908153  161.068692  34.259261  5.177292   951.377075
4           4  1262908153  151.167611  29.910828  4.431568   973.604980
